# Testing Backend Services: BRD Test Cases and Test Data Generation

This notebook tests the two main services in the backend:
- `/generate/brd-testcases`: Generate test cases from Business Requirements Documents
- `/generate/test-data`: Generate test data based on test cases

In [2]:
# Import required libraries
import requests
import json
import time
from IPython.display import display, Markdown, JSON
import pandas as pd
import re

In [3]:
# Define the base URL for the backend API
BASE_URL = "http://localhost:8000"  # Adjust this if your backend runs on a different port
print(f"Testing backend services at {BASE_URL}")

Testing backend services at http://localhost:8000


## 1. Test BRD Test Cases Generation Service

In [4]:
# Test BRD Test Cases Generation
def test_brd_testcases(text=None, file_path=None, provider="ollama", model_name="gemma3:4b", 
                      temperature=0.1, max_tokens=4096, response_format="text"):
    """
    Test the BRD test cases generation endpoint
    """
    url = f"{BASE_URL}/generate/brd-testcases"
    
    files = None
    data = {
        "provider": provider,
        "model_name": model_name,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "response_format": response_format
    }
    
    if text:
        data["text"] = text
    elif file_path:
        files = {"file": open(file_path, "rb")}
    
    try:
        print(f"Sending request to {url}...")
        response = requests.post(url, files=files, data=data)
        
        print(f"Response status: {response.status_code}")
        print(f"Response headers: {dict(response.headers)}")
        
        if response.status_code == 200:
            if response_format == "json":
                result = response.json()
                print("\nJSON Response:")
                display(JSON(result))
                return result
            else:
                result = response.text
                print("\nText Response:")
                #print(result)
                # Display as markdown if it contains markdown content
                display(Markdown(result))
                return result
        else:
            print(f"Error: {response.status_code}")
            print(f"Error message: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {str(e)}")
        return None
    finally:
        if files:
            files["file"].close()

## 2. Test Data Generation Service

In [5]:
# Test Test Data Generation
def test_generate_test_data(test_cases, provider="ollama", model_name="gemma3:4b", 
                           temperature=0.1, response_format="text"):
    """
    Test the test data generation endpoint
    """
    url = f"{BASE_URL}/generate/test-data"
    
    data = {
        "provider": provider,
        "model_name": model_name,
        "temperature": temperature,
        "response_format": response_format,
        "test_cases": test_cases
    }
    
    try:
        print(f"Sending request to {url}...")
        response = requests.post(url, data=data)
        
        print(f"Response status: {response.status_code}")
        print(f"Response headers: {dict(response.headers)}")
        
        if response.status_code == 200:
            if response_format == "json":
                result = response.json()
                print("\nJSON Response:")
                display(JSON(result))
                return result
            else:
                result = response.text
                print("\nText Response:")
                #print(result)
                # Display as markdown if it contains markdown content
                display(Markdown(result))
                return result
        else:
            print(f"Error: {response.status_code}")
            print(f"Error message: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {str(e)}")
        return None

In [6]:
def extract_token_counts(text):
    """
    Extract token counts from response text using regex
    Returns a dictionary with prompt, response, and total token counts
    """
    TOKEN_COUNT_REGEX = r'\[TOKEN_COUNTS:Prompt=(\d+),Response=(\d+),Total=(\d+)\]'
    match = re.search(TOKEN_COUNT_REGEX, text)
    if match:
        return {
            'prompt_tokens': int(match.group(1)),
            'response_tokens': int(match.group(2)),
            'total_tokens': int(match.group(3))
        }
    return None

In [30]:
SDDs=['data/BRDs/315620 - Apple Unbricks Data/SDD_ PPM 315620 Apple Unbricks data V1.0.docx','data/BRDs/311566   Device insurance standalone price enhancement (Receipt)/SDD_312181_AppleCare 60 days post purchase window - CR_V1.1.docx','data/BRDs/311224 AppleCare+ 60 days post purchase window - CR/SDD_312181_AppleCare 60 days post purchase window - CR_V1.1.docx','data/BRDs/307899  Battle XO - Arena Esports Platform Replacement - Drop 1/307899-SDD-BattleXOArenaEsportspartnerreplacementv1.1.docx']

In [31]:
total_tokeans=0
for sdd in SDDs:
    print(f"Processing SDD: {sdd}")
    print("Generating Test Cases...")
    brd_result = test_brd_testcases(file_path=sdd, response_format="text")
    brd_result_tc=extract_token_counts(brd_result)
    total_tokeans+=int(brd_result_tc['total_tokens'])
    time.sleep(2)  # Pause between requests
    if brd_result:
        print("Generating Test Data...")
        brd_result=re.sub('\[TOKEN_COUNTS:Prompt=\d+,Response=\d+,Total=\d+\]', '', brd_result).strip()
        test_data_result = test_generate_test_data(test_cases=brd_result, response_format="text")
        test_data_result_tc=extract_token_counts(test_data_result)
        total_tokeans+=int(test_data_result_tc['total_tokens'])
        time.sleep(2)  # Pause between requests

print("No. of SDDs processed:", len(SDDs))
print(f"Total tokens used: {total_tokeans}")
print(f"Mean tokens per SDD: {total_tokeans/len(SDDs)}")

Processing SDD: data/BRDs/315620 - Apple Unbricks Data/SDD_ PPM 315620 Apple Unbricks data V1.0.docx
Generating Test Cases...
Sending request to http://localhost:8000/generate/brd-testcases...


<>:11: SyntaxWarning: invalid escape sequence '\['
<>:11: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_135069/3026816074.py:11: SyntaxWarning: invalid escape sequence '\['
  brd_result=re.sub('\[TOKEN_COUNTS:Prompt=\d+,Response=\d+,Total=\d+\]', '', brd_result).strip()


Response status: 200
Response headers: {'date': 'Mon, 13 Oct 2025 07:29:12 GMT', 'server': 'uvicorn', 'content-type': 'text/plain; charset=utf-8', 'transfer-encoding': 'chunked'}

Text Response:


Okay, here’s a comprehensive set of test cases generated from the provided solution design document.  I’ve aimed for clarity, detail, and coverage, following all your specified requirements.

**Test Case Table**

| Test Case ID | Category | Test Case Description | Preconditions | Test Steps | Test Data | Expected Result | Priority |
|---|---|---|---|---|---|---|---|
| TC-AUTH-01 | Positive - Device Purchase | Verify 500GB data offer activation after iPhone 17 purchase (Prepaid) | Customer has a new iPhone 17 (Prepaid) | 1. Customer purchases iPhone 17. 2. System automatically triggers offer provisioning. 3. Customer confirms offer acceptance. | iPhone 17 (Prepaid) | 500GB data offer is activated within 48 hours, reflected in account details. | High |
| TC-AUTH-02 | Positive - Device Purchase | Verify 500GB data offer activation after iPhone 17 purchase (Postpaid) | Customer has a new iPhone 17 (Postpaid) | 1. Customer purchases iPhone 17. 2. System automatically triggers offer provisioning. 3. Customer confirms offer acceptance. | iPhone 17 (Postpaid) | 500GB data offer is activated within 48 hours, reflected in account details. | High |
| TC-AUTH-03 | Positive - Offer Provisioning | Verify Offer Activation Timeline | Customer has a new iPhone 17 (Prepaid) | 1. Customer purchases iPhone 17. 2. System automatically triggers offer provisioning. 3. Monitor system logs for offer activation. | iPhone 17 (Prepaid) | Offer is activated within 48 hours of purchase. | High |
| TC-AUTH-04 | Negative - Offer Provisioning - Expiry | Verify Offer Deactivation - Attempt to use after expiry | Offer is active for 30 days. | 1. Customer attempts to use the data offer after 31 days. |  Data Usage | Offer is deactivated, and user receives an error message. | High |
| TC-AUTH-05 | Negative - Offer Provisioning - Invalid TAC | Verify Offer Deactivation - Invalid TAC | Offer is active for 30 days. | 1. Customer attempts to use the data offer with an invalid TAC code. | Invalid TAC Code | Offer is deactivated, and user receives an error message. | High |
| TC-AUTH-06 | Edge Case - First Usage | Verify First Usage Activation | Customer has a new iPhone 17 (Prepaid) | 1. Customer purchases iPhone 17. 2. Customer makes their first data usage. | iPhone 17 (Prepaid) | Data offer is activated immediately upon first usage. | High |
| TC-AUTH-07 | Negative - Input Validation - Email Format | Verify Email Format Validation | Customer is entering email address for offer acceptance | 1. Customer enters an email address with an invalid format (e.g., missing @ symbol). | Invalid Email Format | System displays an error message indicating the email format is incorrect. | Medium |
| TC-AUTH-08 | Negative - Input Validation - TAC Length | Verify TAC Length Validation | Customer is entering TAC code for offer acceptance | 1. Customer enters a TAC code with an incorrect length. | Incorrect TAC Length | System displays an error message indicating the TAC code length is incorrect. | Medium |
| TC-AUTH-09 | Edge Case - Multiple Device Purchases (Prepaid) | Verify Single Offer Activation | Customer purchases iPhone 17 (Prepaid) and then another iPhone 17 (Prepaid) | 1. Customer purchases iPhone 17 (Prepaid). 2. Customer purchases another iPhone 17 (Prepaid). | iPhone 17 (Prepaid) x2 | Offer is activated only once per account. | High |
| TC-AUTH-10 | Edge Case - Multiple Device Purchases (Postpaid) | Verify Single Offer Activation | Customer purchases iPhone 17 (Postpaid) and then another iPhone 17 (Postpaid) | 1. Customer purchases iPhone 17 (Postpaid). 2. Customer purchases another iPhone 17 (Postpaid). | iPhone 17 (Postpaid) x2 | Offer is activated only once per account. | High |
| TC-AUTH-11 | Negative - System - No Usage | Verify Offer Deactivation - No Usage | Offer is active for 30 days. | 1. Customer does not make any data usage. |  | Offer is deactivated after 30 days. | Medium |
| TC-AUTH-12 | Positive - SMS Notification - Purchase | Verify SMS Notification - Purchase | Customer purchases iPhone 17 (Prepaid) | 1. Customer purchases iPhone 17 (Prepaid). 2. System sends SMS notification confirming offer activation. | iPhone 17 (Prepaid) | Customer receives SMS notification confirming offer activation. | High |
| TC-AUTH-13 | Positive - SMS Notification - Activation | Verify SMS Notification - Activation | Offer is active for 30 days. | 1. Customer makes their first data usage. 2. System sends SMS notification confirming data usage. |  | Customer receives SMS notification confirming data usage. | High |
| TC-AUTH-14 | Negative - System - System Error | Verify Error Handling | Offer is active for 30 days. | 1. Simulate a system error during offer activation. |  | System logs the error, and the offer is not activated. | Low |
| TC-AUTH-15 | Positive - Mapping Table Configuration | Verify Mapping Table Configuration | Offer is active for 30 days. | 1. Verify the mapping table configuration for Usage Inquiry. |  | The mapping table is correctly configured. | Medium |
| TC-AUTH-16 | Positive -  Offer ID Configuration | Verify Offer ID Configuration | Offer is active for 30 days. | 1. Verify the mapping table configuration for Usage Inquiry. |  | The mapping table is correctly configured. | Medium |

**Summary of Test Cases:**

*   **Total Test Cases Generated:** 16
*   **Categories:**
    *   Positive - Offer Activation & Usage (6)
    *   Negative - Input Validation & Error Handling (5)
    *   Edge Cases - First Usage & Multiple Device Purchases (3)
    *   Mapping Table Configuration (2)

**Note:** This table represents a starting point.  Further refinement and expansion would be necessary based on deeper analysis of the solution design and potential risks.  It's crucial to prioritize these test cases based on the overall project risk assessment.

[TOKEN_COUNTS:Prompt=3951,Response=1362,Total=5313]

Generating Test Data...
Sending request to http://localhost:8000/generate/test-data...
Response status: 200
Response headers: {'date': 'Mon, 13 Oct 2025 07:29:50 GMT', 'server': 'uvicorn', 'content-type': 'text/plain; charset=utf-8', 'transfer-encoding': 'chunked'}

Text Response:


...

Processing SDD: data/BRDs/307899  Battle XO - Arena Esports Platform Replacement - Drop 1/307899-SDD-BattleXOArenaEsportspartnerreplacementv1.1.docx
Generating Test Cases...
Sending request to http://localhost:8000/generate/brd-testcases...
Response status: 200
Response headers: {'date': 'Mon, 13 Oct 2025 07:33:06 GMT', 'server': 'uvicorn', 'content-type': 'text/plain; charset=utf-8', 'transfer-encoding': 'chunked'}

Text Response:


Okay, here's the generated test case suite based on the provided Solution Design Document.  I've aimed for thorough coverage, clear descriptions, and a structured format.

**Test Case Suite: Battle XO/Arena Esports Platform Replacement - Drop 1 Version 1.1**

| Test Case ID | Category | Test Case Description | Preconditions | Test Steps | Test Data | Expected Result |
|---|---|---|---|---|---|---|
| TC-REG-01 | Registration - Successful | Verify successful registration via B2C Portal. | User navigates to B2C Portal. | 1. Enter valid email address. 2. Enter valid password. 3. Confirm password. 4. Click "Join Now". | email: testuser@example.com, Password: P@sswOrd123 | User is successfully registered and redirected to the Arena Esports platform. Confirmation email is received. |
| TC-REG-02 | Registration - Invalid Email | Verify registration fails with invalid email format. | User navigates to B2C Portal. | 1. Enter invalid email address (e.g., test@example). 2. Enter valid password. 3. Confirm password. 4. Click "Join Now". | email: test@example, Password: P@sswOrd123 | Error message: "Invalid email format. Please enter a valid email address." |
| TC-REG-03 | Registration - Password Mismatch | Verify registration fails with password mismatch. | User navigates to B2C Portal. | 1. Enter valid email address. 2. Enter valid password. 3. Enter a different password. 4. Confirm password. 5. Click "Join Now". | email: testuser@example.com, Password: P@sswOrd123, Confirm Password: P@sswOrd124 | Error message: "Passwords do not match." |
| TC-REG-04 | Registration - Password Complexity | Verify registration fails with password not meeting complexity requirements. | User navigates to B2C Portal. | 1. Enter valid email address. 2. Enter a password that does not meet complexity requirements (e.g., too short, no special characters). 3. Confirm password. 4. Click "Join Now". | email: testuser@example.com, Password: Password, Confirm Password: Password | Error message: "Password must meet complexity requirements." |
| TC-SUB-01 | Subscription - Successful | Verify successful subscription to an Arena Esports game. | User is successfully registered and logged in. | 1. Navigate to the Arena Esports platform. 2. Select a game. 3. Confirm the subscription. | Game: Arena Esports Game X | User is successfully subscribed to the selected game. Confirmation message is displayed. |
| TC-SUB-02 | Subscription - Invalid Link | Verify subscription fails with an expired or invalid redirection link. | User is successfully registered and logged in. | 1. Navigate to the Arena Esports platform. 2. Click on a link that redirects to a non-functional page. | Link:  https://arenaesports.ae/invalid | Error message: "Redirect link is invalid. Please try again." or Page does not load. |
| TC-PAY-01 | Payment - Successful | Verify successful payment via credit card. | User is successfully subscribed to an Arena Esports game. | 1. Initiate the payment process. 2. Enter valid credit card details. 3. Confirm the payment. | Credit Card: Valid Card Details | Payment is processed successfully. Confirmation message is displayed. |
| TC-PAY-02 | Payment - Invalid Credit Card | Verify payment fails with an invalid credit card. | User is successfully subscribed to an Arena Esports game. | 1. Initiate the payment process. 2. Enter invalid credit card details. 3. Confirm the payment. | Credit Card: Invalid Card Details | Error message: "Invalid credit card details. Please try again." |
| TC-PAY-03 | Payment - Insufficient Funds | Verify payment fails due to insufficient funds. | User is successfully subscribed to an Arena Esports game. | 1. Initiate the payment process. 2. Enter valid credit card details that have insufficient funds. 3. Confirm the payment. | Credit Card: Card with insufficient funds | Error message: "Payment declined due to insufficient funds." |
| TC-SEC-01 | Security - Redirection | Verify the redirection from B2C Portal to Arena Esports platform is secure. | User is successfully registered and logged in. | 1. Navigate to the Arena Esports platform. 2. Click on the redirection link. | Link: https://arenaesports.ae | User is redirected to the Arena Esports platform without any data being intercepted. |
| TC-INF-01 | Integration - EPG Access | Verify BXO platform integrates correctly with EPG. | User is successfully subscribed to an Arena Esports game. | 1. Navigate to the Arena Esports platform. 2. Initiate the payment process. |  | Payment is processed correctly through the EPG. |
| TC-CAP-01 | Capacity - High Traffic | Simulate high traffic to the BXO platform. |  | Monitor server performance during peak usage. |  | System remains stable and responsive. |
| TC-NON-01 | Non-Functional - Response Time | Measure the response time of the BXO platform. |  |  |  | Response time is within acceptable limits (e.g., < 3 seconds). |

**Total Number of Test Cases Generated: 18**

**Categorization of Test Cases:**

*   **Positive (Happy Path):** 6
*   **Negative Paths (Error Handling):** 6
*   **Edge Cases/Boundary Conditions:** 3
*   **Integration/System Testing:** 3
*   **Performance/Capacity Testing:** 1

**Note:** This is a starting point.  Further refinement and expansion of these test cases would be necessary based on a deeper understanding of the system's architecture, specific requirements, and potential risks.  It's also crucial to create test data sets that cover a wide range of scenarios.

[TOKEN_COUNTS:Prompt=2467,Response=1289,Total=3756]

Generating Test Data...
Sending request to http://localhost:8000/generate/test-data...
Response status: 200
Response headers: {'date': 'Mon, 13 Oct 2025 07:33:38 GMT', 'server': 'uvicorn', 'content-type': 'text/plain; charset=utf-8', 'transfer-encoding': 'chunked'}

Text Response:


Okay, this is a fantastic starting point! You've done a great job of outlining a comprehensive test suite. Let's build upon this with a more detailed and robust synthetic test data table. I'll expand on the existing data, add more realistic values, and incorporate edge cases and boundary conditions. I'll also add some notes on data types and potential ranges.

**Synthetic Test Data Table - Battle XO/Arena Esports Platform Replacement - Drop 1 Version 1.1 (Expanded)**

| Test Case ID | Category | Test Case Description | Preconditions | Test Steps | Test Data | Expected Result | Data Type | Notes |
|---|---|---|---|---|---|---|---|---|
| TC-REG-01 | Registration - Successful | Verify successful registration via B2C Portal. | User navigates to B2C Portal. | 1. Enter valid email address. 2. Enter valid password. 3. Confirm password. 4. Click "Join Now". | email: testuser123@example.com, Password: P@sswOrd123, Confirm Password: P@sswOrd123 | User is successfully registered and redirected to the Arena Esports platform. Confirmation email is received. | String, String, String |  Include a realistic email domain (e.g., @example.com) |
| TC-REG-02 | Registration - Invalid Email | Verify registration fails with invalid email format. | User navigates to B2C Portal. | 1. Enter invalid email address (e.g., test@example). 2. Enter valid password. 3. Confirm password. 4. Click "Join Now". | email: test@example, Password: P@sswOrd123, Confirm Password: P@sswOrd123 | Error message: "Invalid email format. Please enter a valid email address." | String, String, String |  Vary the invalid email format (e.g., missing @, invalid characters). |
| TC-REG-03 | Registration - Password Mismatch | Verify registration fails with password mismatch. | User navigates to B2C Portal. | 1. Enter valid email address. 2. Enter valid password. 3. Enter a different password. 4. Confirm password. 5. Click "Join Now". | email: testuser123@example.com, Password: P@sswOrd123, Confirm Password: P@sswOrd124 | Error message: "Passwords do not match." | String, String, String |  Ensure the difference is significant (e.g., one character different). |
| TC-REG-04 | Registration - Password Complexity | Verify registration fails with password not meeting complexity requirements. | User navigates to B2C Portal. | 1. Enter valid email address. 2. Enter a password that does not meet complexity requirements (e.g., too short, no special characters). 3. Confirm password. 4. Click "Join Now". | email: testuser123@example.com, Password: Password, Confirm Password: Password | Error message: "Password must meet complexity requirements." | String, String, String |  Test with passwords below minimum length, without special characters, and without uppercase/lowercase. |
| TC-SUB-01 | Subscription - Successful | Verify successful subscription to an Arena Esports game. | User is successfully registered and logged in. | 1. Navigate to the Arena Esports platform. 2. Select a game. 3. Confirm the subscription. | Game: Arena Esports Game X | User is successfully subscribed to the selected game. Confirmation message is displayed. | String |  Game name should be a realistic title. |
| TC-SUB-02 | Subscription - Invalid Link | Verify subscription fails with an expired or invalid redirection link. | User is successfully registered and logged in. | 1. Navigate to the Arena Esports platform. 2. Click on a link that redirects to a non-functional page. | Link: https://arenaesports.ae/invalid | Error message: "Redirect link is invalid. Please try again." or Page does not load. | String |  Create a deliberately broken link. |
| TC-PAY-01 | Payment - Successful | Verify successful payment via credit card. | User is successfully subscribed to an Arena Esports game. | 1. Initiate the payment process. 2. Enter valid credit card details. 3. Confirm the payment. | Credit Card: Valid Card Details (e.g., Visa ending in 1234) | Payment is processed successfully. Confirmation message is displayed. | String, String, String, String, String |  Use a test credit card number (ensure it's not a real card!). |
| TC-PAY-02 | Payment - Invalid Credit Card | Verify payment fails with an invalid credit card. | User is successfully subscribed to an Arena Esports game. | 1. Initiate the payment process. 2. Enter invalid credit card details. 3. Confirm the payment. | Credit Card: Invalid Card Details (e.g., incorrect number, expired) | Error message: "Invalid credit card details. Please try again." | String, String, String, String, String |  Vary the invalid details to cover common errors. |
| TC-PAY-03 | Payment - Insufficient Funds | Verify payment fails due to insufficient funds. | User is successfully subscribed to an Arena Esports game. | 1. Initiate the payment process. 2. Enter valid credit card details that have insufficient funds. 3. Confirm the payment. | Credit Card: Card with insufficient funds (simulated) | Error message: "Payment declined due to insufficient funds." | String, String, String, String, String |  Simulate a card with a low balance. |
| TC-SEC-01 | Security - Redirection | Verify the redirection from B2C Portal to Arena Esports platform is secure. | User is successfully registered and logged in. | 1. Navigate to the Arena Esports platform. 2. Click on the redirection link. | Link: https://arenaesports.ae | User is redirected to the Arena Esports platform without any data being intercepted. | String |  Verify the URL is correct and secure (HTTPS). |
| TC-INF-01 | Integration - EPG Access | Verify BXO platform integrates correctly with EPG. | User is successfully subscribed to an Arena Esports game. | 1. Navigate to the Arena Esports platform. 2. Initiate the payment process. |  | Payment is processed correctly through the EPG. |  |  This test requires integration with a mock EPG system. |
| TC-CAP-01 | Capacity - High Traffic | Simulate high traffic to the BXO platform. |  | Monitor server performance during peak usage. |  | System remains stable and responsive. |  |  Requires tools to simulate high traffic. |
| TC-NON-01 | Non-Functional - Response Time | Measure the response time of the BXO platform. |  |  |  | Response time is within acceptable limits (e.g., < 3 seconds). |  |  Use performance testing tools. |

**Key Improvements & Considerations:**

*   **Data Types:** Added data types (String, Integer, etc.) to clarify the expected data format.
*   **Realistic Values:**  Included more realistic values for email addresses, credit card numbers (using test data), and game names.
*   **Edge Cases:** Expanded on edge cases, such as invalid email formats and password complexity.
*   **Boundary Conditions:** Added examples for boundary conditions (e.g., minimum password length).
*   **Mock Data:**  Emphasized the use of mock data for sensitive information like credit card numbers.
*   **Integration Testing:**  Added a placeholder for integration testing with the EPG.
*   **Performance Testing:** Included a placeholder for capacity/performance testing.

**Next Steps:**

*   **Tooling:**  Consider using a test data generation tool to automate the creation of this data.
*   **Data Masking:** Implement data masking techniques to protect sensitive information.
*   **Test Data Management:** Establish a process for managing and maintaining the test data.

This expanded table provides a much more robust foundation for your testing efforts.  Remember to tailor the data to the specific requirements of your application.  Let me know if you'd like me to elaborate on any particular aspect or add more test cases!

[TOKEN_COUNTS:Prompt=1328,Response=1750,Total=3078]

No. of SDDs processed: 4
Total tokens used: 29527
Mean tokens per SDD: 7381.75


## 3. Knowledge Base Query Service

In [8]:
def knowledge_base_query(query, provider="ollama", model_name="gemma3:4b", 
                           temperature=0.1, response_format="text"):
    """
    Test the test data generation endpoint
    """
    url = f"{BASE_URL}/kb/generate"
    
    data = {
        "provider": provider,
        "model_name": model_name,
        "temperature": temperature,
        "response_format": response_format,
        "query": query
    }
    
    try:
        print(f"Sending request to {url}...")
        response = requests.post(url, data=data)
        
        print(f"Response status: {response.status_code}")
        print(f"Response headers: {dict(response.headers)}")
        
        if response.status_code == 200:
            if response_format == "json":
                result = response.json()
                print("\nJSON Response:")
                display(JSON(result))
                return result
            else:
                result = response.text
                print("\nText Response:")
                #print(result)
                # Display as markdown if it contains markdown content
                display(Markdown(result))
                return result
        else:
            print(f"Error: {response.status_code}")
            print(f"Error message: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {str(e)}")
        return None

### 2 Ways of Retriving information
**1** retreiver detects 'test case/s' or 'testcase/s' in query --> Associated tests to sematic search is retrevied from Mongodb  
**2** else --> Normal semantic search and content retreived

In [9]:
kb_result=knowledge_base_query("generate test cases for gaming service")

Sending request to http://localhost:8000/kb/generate...
Response status: 200
Response headers: {'date': 'Wed, 05 Nov 2025 10:39:13 GMT', 'server': 'uvicorn', 'content-type': 'text/plain; charset=utf-8', 'Transfer-Encoding': 'chunked'}

Text Response:


Okay, here's the generated test case table based on the provided input document, formatted as requested.

**Generated Test Cases**

| Test Case ID | Category | Test Case Description | Preconditions | Test Steps | Expected Result | Priority |
|---|---|---|---|---|---|---|
| TC-AUTH-01 | Positive - Initial Purchase | Purchase gaming service from Areana eSports using Credit card and verify payment is successful from Etisalat EPG | Account: Etisalat Customer with Any type of account, Not registered on Arena Esports | 1. Visit B2C portal -> Digital Products -> Gaming -> Arena Esports -> Choose / Click product -Join Now. | 1. On Clicking link “Join Now”, Verify Customer should the Re-directed to a new URL https://arenaesports.ae to point to new platform (BXO). 2. Complete Registeration on Platform by providing required information on the platform (name, Email, Password). 3. After successful registration Login to arena esport with login credentials set. 4. Select any game/service on platform that require one time payment. 5. Verify customer is provided with payment option along with Credit/Debit card. 6. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 7. Enter credit card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge (One-time payment) from credit card. 8. Game/Service selected for purchase should be successful. | High |
| TC-AUTH-02 | Positive - International Credit Card | Purchase gaming service from Areana eSports using International Credit card and verify payment is successful from Etisalat EPG | Account: Etisalat Customer with Any type of account, Not registered on Arena Esports | 1. Goto Arena Esports Website https://arenaesports.ae | 1. Click on Register 2. Complete Registeration on Platform by providing required information on the platform (name, Email, Password) 3. After successful registration Login to arena esport with login credental set. 4. Select any game /service on platform that require one time payment. 5. Verify customer is provided with payment option along with Credit/Debit card. 6. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 7. Enter International credit card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge ( One-time payment) from credit card. 8. Game / Service selected for purchase sould be should be successful | High |
| TC-AUTH-03 | Positive - Non-Etisalat Customer Credit Card | Verify any NON etisalat customer is able to Purchase gaming service from Areana eSports using Credit card and verify payment is successful from Etisalat EPG | Account: NON Etisalat, Not registered on Arena Esports | 1. Visit B2C portal -> Digital Products -> Gaming -> Arena Esports -> Choose / Click product -Join Now. | 1. On Clicking link “Join Now”, Verify Customer should the Re-directed to a new URL https://arenaesports.ae to point to new platform (BXO). 2. Complete Registeration on Platform by providing required information on the platform (name, Email, Password). 3. After successful registration Login to arena esport with login credental set. 4. Select any game/service on platform that require one time payment. 5. Verify customer is provided with payment option along with Credit/Debit card. 6. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 7. Enter credit card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge (One-time payment) from credit card. 8. Game/Service selected for purchase should be successful | High |
| TC-AUTH-04 | Positive - Existing Customer Credit Card | Purchase gaming service from Areana eSports using Credit card and verify payment is successful from Etisalat EPG | Account: Existing Customer, Registered on Arena Esports, Have Login credentials | 1. Goto Arena Esports Website https://arenaesports.ae and Login with credentials | 1. Select any game /service on platform that require one time payment. 2. Verify customer is provided with payment option along with Credit/Debit card. 3. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 4. Enter card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge ( One-time payment) from credit card. 5. Game / Service selected for purchase sould be should be successful | High |
| TC-AUTH-05 | Positive - Existing Postpaid Customer Credit Card | Purchase gaming service from Areana eSports using Credit card and verify payment is successful from Etisalat EPG | Account: Existing Postpaid, Not registered on Arena Esports | 1. Goto Arena Esports Website https://arenaesports.ae and Login with credentials | 1. Select  any game /service on platform that require one time payment. 2. Verify customer is provided with payment option along with Credit/Debit card. 3. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 4. Enter card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge ( One-time payment) from credit card. 5. Game / Service selected for purchase sould be should be successful | High |
| TC-AUTH-06 | Positive - Existing Postpaid Customer Debit Card | Purchase gaming service from Areana eSports using Debit card and verify payment is successful from Etisalat EPG | Account: Existing Postpaid, Not registered on Arena Esports | 1. Goto Arena Esports Website https://arenaesports.ae and Login with credentials | 1. Select  any game /service on platform that require one time payment. 2. Verify customer is provided with payment option along with Credit/Debit card. 3. Select Debit card option and verify customer is redirect to Etisalat Payment page for payment. 4. Enter Debit card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge ( One-time payment) from Debit card. 5. Game / Service selected for purchase sould be should be successful | High |
| TC-AUTH-07 | Positive - Existing Postpaid Customer Debit Card | Purchase gaming service from Areana eSports using Debit card and verify payment is successful from Etisalat EPG | Account: Existing Postpaid, Not registered on Arena Esports | 1. Goto Arena Esports Website https://arenaesports.ae and Login with credentials | 1. Select  any game /service on platform that require one time payment. 2. Verify customer is provided with payment option along with Credit/Debit card. 3. Select Debit card option and verify customer is redirect to Etisalat Payment page for payment. 4. Enter Debit card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge ( One-time payment) from Debit card. 5. Game / Service selected for purchase sould be should be successful | High |
| TC-AUTH-08 | Positive - Existing Customer Debit Card | Purchase gaming service from Areana eSports using Debit card and verify payment is successful from Etisalat EPG | Account: Existing Customer, Registered on Arena Esports, Have Login credentials | 1. Goto Arena Esports Website https://arenaesports.ae and Login with credentials | 1. Select  any game /service on platform that require one time payment. 2. Verify customer is provided with payment option along with Credit/Debit card. 3. Select Debit card option and verify customer is redirect to Etisalat Payment page for payment. 4. Enter card details and perform authentication using OTP and verify Payment should be successful. - Note: Etisalat Payment Gateway access to new BxO platform and Deduct subscription charge ( One-time payment) from Debit card. 5. Game / Service selected for purchase sould be should be successful | High |
| TC-AUTH-09 | Negative - Incorrect Credit Card Details | Initiate Purchase gaming service from Areana eSports using Credit card and verify when incorrect card details are entered payment should be unsuccessfull  from Etisalat EPG | Account: Etisalat Customer with Any type of account, Not registered on Arena Esports | 1. Visit B2C portal -> Digital Products -> Gaming -> Arena Esports -> Choose / Click product -Join Now. | 1. On Clicking link “Join Now”, Verify Customer should the Re-directed to a new URL https://arenaesports.ae to point to new platform (BXO). 2. Complete Registeration on Platform by providing required information on the platform (name, Email, Password). 3. After successful registration Login to arena esport with login credentials set. 4. Select any game/service on platform that require one time payment. 5. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 6. Enter Incorrect card details and perform authentication using OTP and verify Payment should be unsuccessfull and no gaming service should be purchased. | High |
| TC-AUTH-10 | Negative - Incorrect Debit Card Details | Initiate Purchase gaming service from Areana eSports using Debit card and verify when incorrect card details are entered payment should be unsuccessfull  from Etisalat EPG | Account: Non Etisalat, Not registered on Arena Esports | 1. Visit B2C portal -> Digital Products -> Gaming -> Arena Esports -> Choose / Click product -Join Now. | 1. On Clicking link “Join Now”, Verify Customer should the Re-directed to a new URL https://arenaesports.ae to point to new platform (BXO). 2. Complete Registeration on Platform by providing required information on the platform (name, Email, Password). 3. After successful registration Login to arena esport with login credentials set. 4. Select any game/service on platform that require one time payment. 5. Select Debit card option and verify customer is redirect to Etisalat Payment page for payment. 6. Enter Incorrect Debit card details and perform authentication using OTP and verify Payment should be unsuccessfull and no gaming service should be purchased. | High |
| TC-AUTH-11 | Reporting - Payment Details | Verify all Payment completed on Arena Esports using Etisalat EPG all transaction details should be added in Cash Book | Account: Etisalat Customer with Any type of account, Not registered on Arena Esports | 1. Visit B2C portal -> Digital Products -> Gaming -> Arena Esports -> Choose / Click product -Join Now. | 1. On Clicking link “Join Now”, Verify Customer should the Re-directed to a new URL https://arenaesports.ae to point to new platform (BXO). 2. Complete Registeration on Platform by providing required information on the platform (name, Email, Password). 3. After successful registration Login to arena esport with login credental set. 4. Select any game/service on platform that require one time payment. 5. Verify customer is provided with payment option along with Credit/Debit card. 6. Select Credit card option and verify customer is redirect to Etisalat Payment page for payment. 7. Enter credit card details and perform authentication using OTP and verify Payment should be successful. 8. Verify all Payment completed on Arena Esports using Etisalat EPG all transaction details should be added in Cash Book | Medium |

**Summary:**

Generated 12 test cases: 4 Positive, 5 Negative, 3 Edge Cases.


[TOKEN_COUNTS:Prompt=4482,Response=2631,Total=7113]

In [10]:
kb_result=knowledge_base_query("tell me about Etisalat arena")

Sending request to http://localhost:8000/kb/generate...
Response status: 200
Response headers: {'date': 'Wed, 05 Nov 2025 10:40:32 GMT', 'server': 'uvicorn', 'content-type': 'text/plain; charset=utf-8', 'Transfer-Encoding': 'chunked'}

Text Response:


Okay, here’s a comprehensive analysis of the provided context concerning “Etisalat Arena,” incorporating all relevant details and synthesizing information for a thorough understanding.

**Understanding the Context: Etisalat Arena – A Focus on Game Subscriptions & IT Integrations**

The primary focus of this document is the integration of Arena Esports (likely a competitive gaming platform) with Etisalat, a major telecommunications provider in the region. It’s not simply about “Etisalat Arena” as a physical location, but rather a strategic project to enable Etisalat customers to subscribe to and participate in the Arena Esports platform.

**Key Components & Details:**

1.  **Project Scope: Game Subscriptions via EPG:**
    *   The core activity is facilitating the subscription of games within the Arena Esports platform *through* the Electronic Game Promotion (EPG) system. This suggests a system where users can purchase game access or services related to the platform via Etisalat’s channels.
    *   The document highlights the need for discussions with Security Teams regarding the integration, indicating potential security considerations related to user accounts and data.

2.  **Etisalat’s Role & Integration Requirements:**
    *   **Vendor App Publishing:** A critical aspect is the potential for Etisalat to host or manage vendor applications (likely related to the Arena Esports platform) under Etisalat accounts. This implies a technical integration where the Arena Esports platform’s applications are hosted and managed through Etisalat’s infrastructure.
    *   **IT System Updates:** The document outlines specific IT system updates required, including:
        *   **BR01:** “To be discussed with Security Teams No CIT Impacts” – This indicates a requirement for security reviews and approvals before any integration can proceed. “CIT Impacts” likely refers to “Change Impact Tracking,” a process for assessing the potential effects of changes.
        *   **BR02:** “Product Owner to check with Digital Product Owners on Vendor App publishing under Etisalat account No CIT Impacts” – This specifies a workflow for managing vendor applications, requiring coordination between product owners and digital product owners.
        *   **BR03:** “To remove Swarmio integration IP addresses from the IT systems for Arena eSports No CIT Impacts” – This is a technical requirement, suggesting that Swarmio (likely a monitoring and observability tool) is being integrated, and its IP addresses need to be removed from Arena Esports’ IT systems to maintain security and compliance.

3.  **System Involvement:**
    *   **B2C & EPG:** The project is centered around the B2C (Business-to-Consumer) channel and the EPG system. This suggests a consumer-facing application or service.
    *   **Digital Product Owners:** The involvement of Digital Product Owners highlights the complexity of the integration, requiring expertise in digital product management.

4.  **Dependencies & Risks:**
    *   **New Vendor Connectivity:** The availability of new vendor connectivity is a critical dependency. This means that the infrastructure and network connections need to be in place to support the integration.
    *   **Security Considerations:** The repeated emphasis on security (Security Teams, CIT Impacts, Swarmio IP addresses) underscores the importance of robust security measures.

**Relationships & Connections:**

*   The EPG system is the *enabler* – it’s the mechanism through which users subscribe to Arena Esports games.
*   Etisalat’s infrastructure provides the *platform* for this subscription process.
*   The integration requires close collaboration between Etisalat’s various teams (Product Owners, Digital Product Owners, Security Teams) and Arena Esports.

**Missing Information & Considerations:**

*   **Specific Game Titles:** The document doesn’t detail which games are being offered through the Arena Esports platform.
*   **Technical Specifications:** There are no technical specifications provided regarding the EPG system, the Arena Esports platform, or the integration architecture.
*   **User Interface/Experience:** There’s no information about the user interface or the overall customer experience.

**In summary, “Etisalat Arena” in this context represents a strategic partnership to bring competitive gaming to Etisalat customers through a subscription-based service, requiring significant technical and operational integration.**

Do you want me to delve deeper into a specific aspect of this information, such as the technical requirements, the potential risks, or the roles involved?

[TOKEN_COUNTS:Prompt=3353,Response=916,Total=4269]

# Knowledge Base (KB) Tab Flowchart

The Knowledge Base tab allows users to upload documents and use them to generate test cases using AI.

```mermaid
flowchart TD
    A[KB Tab Opened] --> B{User Action}
    B --> C[Upload Knowledge Base ZIP File]
    B --> D[Query Knowledge Base]
    
    C --> E[Select ZIP File with Documents]
    E --> F[Upload File to Backend /kb/upload-zip]
    F --> G[Backend Extracts ZIP File]
    G --> H[Process Documents with Knowledge Base Pipeline]
    H --> I[Process BRD/SDD Documents with DoclingReader]
    I --> J[Parse Excel Files for UAT Test Scenarios]
    J --> K[Store UAT Test Scenarios in MongoDB]
    K --> L[Generate Vector Embeddings]
    L --> M[Store in Vector Index Qdrant/Simple]
    M --> N[Knowledge Base Updated]
    
    D --> O[Enter Query in Text Field]
    O --> P[Send Query to Backend /kb/generate]
    P --> Q[Load Knowledge Base Index]
    Q --> R[Query Vector Index for Relevant Documents]
    R --> Y{Query contains 'test case'?}
    Y -->|Yes| S[Retrieve UAT Test Scenarios from MongoDB using ID]
    Y -->|No| Z[Retrieve Context based on Semantic Similarity]
    S --> T[Generate Response using LLM with UAT Context]
    Z --> AA[Generate Response using LLM with Semantic Context]
    T --> U[Return Generated Test Cases]
    AA --> U
    U --> V[Display Results in UI]
    
    N --> W{User wants to query?}
    W -->|Yes| D
    W -->|No| X[End Session]
    V --> W
```

## Description of KB Tab Flow

The Knowledge Base tab functionality includes two main workflows:

1. **Upload Knowledge Base Documents**:
   - User uploads a ZIP file containing documents (BRDs, SDDs, Excel files with test cases)
   - Backend extracts the ZIP file via `/kb/upload-zip` endpoint
   - Documents are processed through the knowledge base pipeline
   - BRD/SDD documents are processed with DoclingReader for PDF, DOCX, and other formats
   - Excel files are parsed specifically for UAT test scenarios
   - UAT test scenarios are stored separately in MongoDB with reference IDs
   - Content is stored in a vector index (Qdrant or Simple) for semantic search
   - Vector embeddings are generated using configured embedding models

2. **Query Knowledge Base**:
   - User enters a query in the text field
   - Query is sent to the backend `/kb/generate` endpoint
   - Backend loads the knowledge base index
   - Vector index is queried for relevant documents
   - System checks if query contains 'test case' keyword:
     - If 'test case' is detected: UAT test scenarios are retrieved from MongoDB using stored reference IDs
     - If 'test case' is not detected: Context is retrieved based on semantic similarity
   - Response is generated using the LLM based on the appropriate retrieved information
   - Results are displayed in the UI with proper formatting

The knowledge base uses LlamaIndex, Qdrant vector store, MongoDB, and Docling for document processing to store and retrieve relevant information for generating test cases.